Code von https://github.com/Bengemon825/TF_Object_Detection2020

Installation/Update von Protobuf, da es gelegentlich hier zu Problemen kommt

In [ ]:
!pip install --upgrade pip
!pip install --upgrade protobuf

     |████████████████████████████████| 1.5MB 8.3MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 1.3 MB 9.9 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


Installation von Tensorflow 1.15 und Numpy

In [ ]:
%tensorflow_version 1.15
import tensorflow as tf
print(tf.__version__)

!pip install numpy

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
1.15.2


Set the model name to train

In [ ]:
model_name = "tf_API_data1_v01"
#model_name = "tf_API_data2_v01"

Check GPU Status

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


Weitere Informationen zur GPU 

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7409 sha256=5af2a29d554809837b078f456ef0918f46b3cee7f6df911f5fa03ab330e6aeed
  Stored in directory: /root/.cache/pip/wheels/79/c1/b2/b6fc2647f693a084da25e1d31328ab3dbb565cc58fea37e973
Successfully built gputil
Gen RAM Free: 12.5 GB  | Proc size: 473.8 MB
GPU RAM Free: 15813MB | Used: 317MB | Util   2% | Total 16130MB


Mounten der Google Drive Ordner

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# change to working tensorflow directory on the drive
%cd '/content/gdrive/My Drive/objectDetection/models/'


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/objectDetection/models



Installation via protobuf und kompilieren, installieren von setup.py

In [1]:
!pip install Cython
%cd /content/gdrive/My Drive/objectDetection/models/research/
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/objectDetection/models/research/:/content/gdrive/My Drive/objectDetection/models/research/slim'

!python setup.py build
!python setup.py install

[Errno 2] No such file or directory: '/content/gdrive/My Drive/objectDetection/models/research/'
/content
object_detection/protos/*.proto: No such file or directory
python3: can't open file 'setup.py': [Errno 2] No such file or directory
python3: can't open file 'setup.py': [Errno 2] No such file or directory


Verbleibene GPU Zeit überprüfen

In [ ]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', Left/3600)

Time remaining for this session is:  11.790788708461655



Start Training

In [ ]:
!pip install tf_slim
%cd /content/gdrive/My Drive/objectDetection/models/research/object_detection
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/objectDetection/models/research/:/content/gdrive/My Drive/objectDetection/models/research/slim/'

!python train.py --train_dir=training/ --pipeline_config_path=training/ssd_mobilenet_v1_pets.config --logtostderr

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
I1019 07:53:06.127918 139956544206720 learning.py:512] global step 97704: loss = 1.4037 (0.486 sec/step)
INFO:tensorflow:global step 97705: loss = 1.7215 (0.491 sec/step)
I1019 07:53:06.620782 139956544206720 learning.py:512] global step 97705: loss = 1.7215 (0.491 sec/step)
INFO:tensorflow:global step 97706: loss = 2.0888 (0.475 sec/step)
I1019 07:53:07.097127 139956544206720 learning.py:512] global step 97706: loss = 2.0888 (0.475 sec/step)
INFO:tensorflow:global step 97707: loss = 1.4386 (0.461 sec/step)
I1019 07:53:07.559411 139956544206720 learning.py:512] global step 97707: loss = 1.4386 (0.461 sec/step)
INFO:tensorflow:global step 97708: loss = 2.2389 (0.442 sec/step)
I1019 07:53:08.002921 139956544206720 learning.py:512] global step 97708: loss = 2.2389 (0.442 sec/step)
INFO:tensorflow:global step 97709: loss = 1.5742 (0.457 sec/step)
I1019 07:53:08.461274 139956544206720 learning.py:512] global step 97709: loss

Exportieren des inference Graph

In [ ]:
#  .ckpt needs to be updated every time to match last .ckpt generated
#  .config needs to be updated when changing model
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/ssd_mobilenet_v1_pets.config --trained_checkpoint_prefix training/model.ckpt-100152 --output_directory new_graph

Instructions for updating:
Please use `layer.__call__` method instead.
W1019 08:15:36.949288 139912440366976 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I1019 08:15:38.460057 139912440366976 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I1019 08:15:38.499727 139912440366976 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I1019 08:15:38.539360 139912440366976 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

Ordner in Google Drive als ZIP speichern 

In [ ]:
!zip -r model_graph.zip new_graph

updating: new_graph/ (stored 0%)
  adding: new_graph/model.ckpt.data-00000-of-00001 (deflated 7%)
  adding: new_graph/model.ckpt.index (deflated 67%)
  adding: new_graph/checkpoint (deflated 42%)
  adding: new_graph/model.ckpt.meta (deflated 93%)
  adding: new_graph/frozen_inference_graph.pb (deflated 9%)
  adding: new_graph/saved_model/ (stored 0%)
  adding: new_graph/saved_model/variables/ (stored 0%)
  adding: new_graph/saved_model/saved_model.pb (deflated 9%)
  adding: new_graph/pipeline.config (deflated 69%)
